Jing (Thomas) Zhang

In [101]:
from scipy import stats
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
%matplotlib inline
import matplotlib.pyplot as plt
from functools import reduce
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

## Problem 4a

Clearly describe your evaluation function. What is the high-level motivation? Also talk about what else you tried, what worked and what didn't.

I tried to manually tune it but I was not very lucky and I received many low scores, so I looked at the helper functions in `util.py` and found Counter, which can be essentially used as a vector, and its `__mul__` method can be used as dot product. I basically manually calculated a list of features, most of which are pretty standard (can be directed acquired from `GameState` class):

- score: Current Score
- distFood: Manhattan Distance to the closest food
- nFood: Number of Food left
- distScared: Manhattan Distance to the closest scared ghost
- distGhost: Manhattan Distance to the closest non-scared ghost
- nCap: Number of capsules left
- distCap: Manhattan Distance to the closest capsule
- DisWall: Distance to wall

Then, I created a corresponding weight vector by using another `Counter` with the same keys (and values initialize to random numbers).
Due to time constraint.

In [98]:
X, Y = [], []
with open('./train.txt') as f:
    for line in f:
        Xi, Yi = line.strip().split(':')
        X.append([float(xi.strip()) for xi in Xi.split(',')])
        Y.append(float(Yi))
X = np.array(X)
Y = np.array(Y)

In [104]:
model = LinearRegression()
model.fit(X, Y)
model.score(X, Y)

0.25692277391363116

In [106]:
model.coef_

array([  0.7986124 ,  -0.22948957, -16.75800939,   1.24999436,
        -0.72538631,  -0.38637514,  -0.83274631,  -0.4227622 ])

In [108]:
np.argmax(Y)

1435

In [109]:
X[1435]

array([-1.90090366, -0.00869435, -2.20883162, -2.38793855,  0.8091191 ,
       -1.7459972 , -4.75050352,  2.64918368])